In [55]:
#Finbert model
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [56]:
#Test the model
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
result = classifier("Nuclear war has broken out between Russia and the USA")

result = result
print(result)

Device set to use mps:0


[{'label': 'negative', 'score': 0.7090891003608704}]


In [57]:
#test of fetching news data from the news api
from pygooglenews import GoogleNews
import json
import time 
import pandas as pd

gn = GoogleNews()
search = gn.search('^omx', when='3650d') 
news_data = search['entries']

news_data = pd.DataFrame(news_data)
news_data.to_csv("news_data.csv")

#we want to take only the headlines and the date from the news_data

news_titles = news_data['title']

#Save titles to a file
news_titles.to_csv("news_titles.csv")


In [58]:
#we want to load news_titles and enumerate them
import pandas as pd
import numpy as np

news_titles = pd.read_csv('news_titles.csv')

news_titles.drop('Unnamed: 0', axis=1, inplace=True)

#Show all the titles
news_titles



,title
0,Orbea Orca OMX Custom SRAM Red AXS Build Revie...
1,Early Data for OMX-0407 Fuel Expansion Cohorts...
2,New Drug Could Sustain Oxygen-Starved Hearts -...
3,European Market Activity - Nasdaq
4,Orbea Oiz OMX: First Ride Review - Bikers Edge
...,...
94,Nasdaq to pay $4 mln settlement over apparent ...
95,Sweden stocks lower at close of trade; OMX Sto...
96,Denmark stocks higher at close of trade; OMX C...
97,Sweden stocks higher at close of trade; OMX St...


In [60]:
#Check the titles with FInbert model
scores = []
for title in news_titles['title']:
    result = classifier(title)
    print(result)
    
#combine the scores with the titles
scores = pd.DataFrame(scores)
news_titles['scores'] = scores
news_titles.to_csv("news_titles_with_scores.csv")


[{'label': 'neutral', 'score': 0.9471303224563599}]
[{'label': 'neutral', 'score': 0.9043235182762146}]
[{'label': 'neutral', 'score': 0.7902360558509827}]
[{'label': 'neutral', 'score': 0.9371419548988342}]
[{'label': 'neutral', 'score': 0.9009918570518494}]
[{'label': 'neutral', 'score': 0.7056843638420105}]
[{'label': 'positive', 'score': 0.5953915119171143}]
[{'label': 'neutral', 'score': 0.9326032996177673}]
[{'label': 'neutral', 'score': 0.9295192360877991}]
[{'label': 'neutral', 'score': 0.9356501698493958}]
[{'label': 'neutral', 'score': 0.9410625696182251}]
[{'label': 'neutral', 'score': 0.8756631016731262}]
[{'label': 'neutral', 'score': 0.9228236079216003}]
[{'label': 'neutral', 'score': 0.9373089671134949}]
[{'label': 'neutral', 'score': 0.9244688153266907}]
[{'label': 'positive', 'score': 0.9228377342224121}]
[{'label': 'neutral', 'score': 0.767198383808136}]
[{'label': 'neutral', 'score': 0.7821202278137207}]
[{'label': 'neutral', 'score': 0.9373089671134949}]
[{'label': 

ValueError: Cannot set a DataFrame with multiple columns to the single column scores

In [ ]:
print(classifier("The stock market is going up as fed raises interest rates"))

In [ ]:
#Sectors relevant to OMX30 
#Sectors: Industrials, Financials, Telecom, Consumer Discretionary 
#Crawl news data for these sectors in regards to Sweden and OMX30

gn = GoogleNews()
search_1 = gn.search('Industrials Sweden', when='3650d')
search_2 = gn.search('Financials Sweden', when='3650d')
search_3 = gn.search('Telecom Sweden', when='3650d')
search_4 = gn.search('Consumer Discretionary Sweden', when='3650d')

#Combine the data into one dataframe
news_data = search_1['entries'] + search_2['entries'] + search_3['entries'] + search_4['entries']
news_data = pd.DataFrame(news_data)

#peforme sentiment analysis on the news data
news_titles = news_data['title']
scores = []
for title in news_titles:
    result = classifier(title)
    scores.append(result)
    
#combine the scores with the titles putting the scores first
scores = pd.DataFrame(scores)
news_data['scores'] = scores
news_data.to_csv("news_data_with_scores_sectors.csv")


In [64]:
#Test of predicting stock price
import ModelForecaster as mf

prediction = mf.forecast_stock_price("^OMX")
print(prediction)

[*********************100%***********************]  1 of 1 completed
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use mps:0


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(30, 7), dtype=float32). Expected shape (None, 30, 7), but input has incompatible shape (30, 7)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(30, 7), dtype=float32)
  • training=False
  • mask=None